In [2]:
import numpy as np

In [4]:
cov = np.array([
    [ 1.0000, -0.1949,  0.2583, -0.3933],
    [-0.1949,  1.0000, -0.1188,  0.2919],
    [ 0.2583, -0.1188,  1.0000, -0.3443],
    [-0.3933,  0.2919, -0.3443,  1.0000],
])

In [5]:
c1, up1, do1 = 294288603.77097332, 29726222.606283784, -23672504.108643234
c2, up2, do2 = 1422822752.7824574, 34181144.593426704, -33513316.302916765
c3, up3, do3 = 1267256904.8604925, 34413027.555306196, -32740743.069167614
c4, up4, do4 = 189913774.63168541, 17348726.511133254, -19868612.427323759